In [1]:
%cd /content/drive/MyDrive/My Data Analytics/CMS_ROS01

/content/drive/MyDrive/My Data Analytics/CMS_ROS01


"""
# **Root Cause Analysis** for Revenue Cycle Leakage
Identifies WHY underpayments and denials are occurring

This analysis goes beyond WHAT (leakage amounts) to WHY (root causes)
by examining descriptive columns that explain payment patterns
"""

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

In [3]:
# Load cleaned data
print("Loading data for root cause analysis...")
df_fin = pd.read_csv('carrier_01.csv')

Loading data for root cause analysis...


/tmp/ipython-input-2743790640.py:3: DtypeWarning: Columns (94) have mixed types. Specify dtype option on import or set low_memory=False.
  df_fin = pd.read_csv('carrier_01.csv')


In [4]:
# Convert dates
df_fin['LINE_1ST_EXPNS_DT'] = pd.to_datetime(df_fin['LINE_1ST_EXPNS_DT'])
df_fin['NCH_WKLY_PROC_DT'] = pd.to_datetime(df_fin['NCH_WKLY_PROC_DT'])

print(f"Analyzing {len(df_fin):,} claims for root causes...\n")

Analyzing 174,645 claims for root causes...



In [6]:
# Convert financial columns to numeric
financial_cols = ['LINE_SBMTD_CHRG_AMT', 'LINE_ALOWD_CHRG_AMT',
                  'LINE_NCH_PMT_AMT', 'LINE_PRVDR_PMT_AMT']
for col in financial_cols:
    if col in df_fin.columns:
        df_fin[col] = pd.to_numeric(df_fin[col], errors='coerce')

print(f"Data loaded: {len(df_fin):,} records")


Data loaded: 174,645 records


In [7]:
# ============================================================================
# CALCULATE REQUIRED METRICS (if not already present)
# ============================================================================

print("Calculating metrics...")

# Processing delay
if 'PROCESSING_DELAY_DAYS' not in df_fin.columns:
    df_fin['PROCESSING_DELAY_DAYS'] = (
        df_fin['NCH_WKLY_PROC_DT'] - df_fin['LINE_1ST_EXPNS_DT']
    ).dt.days

# Underpayment
if 'UNDERPAYMENT_AMT' not in df_fin.columns:
    df_fin['UNDERPAYMENT_AMT'] = (
        df_fin['LINE_ALOWD_CHRG_AMT'] - df_fin['LINE_NCH_PMT_AMT']
    )

# Payment flags
if 'ZERO_PAID_FLAG' not in df_fin.columns:
    df_fin['ZERO_PAID_FLAG'] = df_fin['LINE_NCH_PMT_AMT'] == 0

if 'PARTIAL_PAID_FLAG' not in df_fin.columns:
    df_fin['PARTIAL_PAID_FLAG'] = (
        (df_fin['LINE_NCH_PMT_AMT'] > 0) &
        (df_fin['LINE_NCH_PMT_AMT'] < df_fin['LINE_ALOWD_CHRG_AMT'])
    )

if 'FULLY_PAID_FLAG' not in df_fin.columns:
    df_fin['FULLY_PAID_FLAG'] = np.isclose(
        df_fin['LINE_NCH_PMT_AMT'],
        df_fin['LINE_ALOWD_CHRG_AMT'],
        atol=1
    )

print(f"✓ Metrics calculated")
print(f"Analyzing {len(df_fin):,} claims for root causes...\n")

Calculating metrics...
✓ Metrics calculated
Analyzing 174,645 claims for root causes...



## ROOT CAUSE DIMENSION 1: DIAGNOSIS PATTERNS

In [8]:
# Question: Are certain diagnoses associated with higher denials/underpayments?
# ============================================================================

print("="*70)
print("ROOT CAUSE 1: DIAGNOSIS PATTERNS")
print("="*70)

# Primary diagnosis analysis
diagnosis_analysis = df_fin.groupby('PRNCPAL_DGNS_CD').agg({
    'CLM_ID': 'count',
    'UNDERPAYMENT_AMT': ['sum', 'mean'],
    'ZERO_PAID_FLAG': 'mean',
    'PARTIAL_PAID_FLAG': 'mean',
    'LINE_ALOWD_CHRG_AMT': 'sum'
}).round(2)

diagnosis_analysis.columns = ['claim_count', 'total_underpayment', 'avg_underpayment',
                               'zero_paid_rate', 'partial_paid_rate', 'total_allowed']

# Filter for diagnoses with meaningful volume
diagnosis_analysis = diagnosis_analysis[diagnosis_analysis['claim_count'] >= 10]
diagnosis_analysis = diagnosis_analysis.sort_values('total_underpayment', ascending=False)

print("\nTop 10 Diagnoses Driving Underpayment:")
print(diagnosis_analysis.head(10))

# Calculate denial rate by diagnosis
diagnosis_analysis['denial_rate'] = diagnosis_analysis['zero_paid_rate'] * 100

# High-risk diagnoses
high_risk_dx = diagnosis_analysis[
    (diagnosis_analysis['zero_paid_rate'] > 0.7) |  # >70% denial
    (diagnosis_analysis['avg_underpayment'] > diagnosis_analysis['avg_underpayment'].quantile(0.9))
]

print(f"\n🚨 HIGH-RISK DIAGNOSES IDENTIFIED: {len(high_risk_dx)}")
print("\nThese diagnoses have >70% denial rate OR top 10% avg underpayment:")
print(high_risk_dx[['claim_count', 'denial_rate', 'avg_underpayment']].head(10))

# Save diagnosis analysis
diagnosis_analysis.to_csv('results/root_cause_diagnosis.csv')


ROOT CAUSE 1: DIAGNOSIS PATTERNS

Top 10 Diagnoses Driving Underpayment:
                 claim_count  total_underpayment  avg_underpayment  \
PRNCPAL_DGNS_CD                                                      
Z733                   52509          1657595.43             31.57   
Z608                   29466           839580.00             28.49   
I259                    9795           461461.22             47.11   
T7432X                 12109           311767.47             25.75   
Z604                   10755           292543.30             27.20   
R931                    4695           227502.64             48.46   
M5450                   3587            91766.90             25.58   
M7918                   2700            86663.86             32.10   
N184                    3098            81570.58             26.33   
J449                     371            56834.89            153.19   

                 zero_paid_rate  partial_paid_rate  total_allowed  
PRNCPAL_DGNS_CD   

## ROOT CAUSE DIMENSION 2: PLACE OF SERVICE

In [9]:
# Question: Are certain service locations (office, hospital, etc.) problematic?
# ============================================================================

print("\n" + "="*70)
print("ROOT CAUSE 2: PLACE OF SERVICE")
print("="*70)

pos_analysis = df_fin.groupby('LINE_PLACE_OF_SRVC_CD').agg({
    'CLM_ID': 'count',
    'UNDERPAYMENT_AMT': ['sum', 'mean'],
    'ZERO_PAID_FLAG': 'mean',
    'LINE_ALOWD_CHRG_AMT': 'mean',
    'PROCESSING_DELAY_DAYS': 'mean'
}).round(2)

pos_analysis.columns = ['claim_count', 'total_underpayment', 'avg_underpayment',
                        'denial_rate', 'avg_allowed_amt', 'avg_processing_days']

pos_analysis = pos_analysis[pos_analysis['claim_count'] >= 50]  # Minimum volume
pos_analysis['denial_rate'] *= 100
pos_analysis = pos_analysis.sort_values('denial_rate', ascending=False)

print("\nPlace of Service Performance:")
print(pos_analysis)

# Common place of service codes:
# 11 = Office, 21 = Inpatient Hospital, 22 = Outpatient Hospital
# 23 = Emergency Room, 81 = Independent Laboratory

print("\n💡 INSIGHT: Place of Service Impact")
if '11' in pos_analysis.index:
    print(f"   Office (11): {pos_analysis.loc['11', 'denial_rate']:.1f}% denial rate")
if '22' in pos_analysis.index:
    print(f"   Outpatient Hospital (22): {pos_analysis.loc['22', 'denial_rate']:.1f}% denial rate")

pos_analysis.to_csv('results/root_cause_place_of_service.csv')


ROOT CAUSE 2: PLACE OF SERVICE

Place of Service Performance:
                       claim_count  total_underpayment  avg_underpayment  \
LINE_PLACE_OF_SRVC_CD                                                      
34                             268             9978.70             37.23   
11                          142899          4112030.61             28.78   
12                             590             9449.89             16.02   
20                           28611           978375.28             34.20   
31                             649            23282.71             35.87   
22                            1601           353664.89            220.90   

                       denial_rate  avg_allowed_amt  avg_processing_days  
LINE_PLACE_OF_SRVC_CD                                                     
34                            63.0           178.62                 4.03  
11                            62.0           131.83                 4.01  
12                          

## ROOT CAUSE DIMENSION 3: PROVIDER CHARACTERISTICS

In [10]:
# Question: Are certain provider types or locations driving issues?
# ============================================================================

print("\n" + "="*70)
print("ROOT CAUSE 3: PROVIDER PATTERNS")
print("="*70)

# Provider specialty analysis (already done, but let's add state)
state_analysis = df_fin.groupby('PRVDR_STATE_CD').agg({
    'CLM_ID': 'count',
    'UNDERPAYMENT_AMT': 'sum',
    'ZERO_PAID_FLAG': 'mean',
    'LINE_ALOWD_CHRG_AMT': 'mean',
    'PROCESSING_DELAY_DAYS': 'mean'
}).round(2)

state_analysis.columns = ['claim_count', 'total_underpayment', 'denial_rate',
                          'avg_claim_size', 'avg_processing_days']
state_analysis['denial_rate'] *= 100
state_analysis = state_analysis[state_analysis['claim_count'] >= 100]
state_analysis = state_analysis.sort_values('denial_rate', ascending=False)

print("\nTop 10 States by Denial Rate:")
print(state_analysis.head(10))

# Provider type (if available via CARR_LINE_PRVDR_TYPE_CD)
if 'CARR_LINE_PRVDR_TYPE_CD' in df_fin.columns:
    provider_type = df_fin.groupby('CARR_LINE_PRVDR_TYPE_CD').agg({
        'CLM_ID': 'count',
        'ZERO_PAID_FLAG': 'mean',
        'UNDERPAYMENT_AMT': 'mean'
    }).round(3)
    provider_type.columns = ['claims', 'denial_rate', 'avg_underpayment']
    provider_type['denial_rate'] *= 100
    print("\nProvider Type Analysis:")
    print(provider_type)

state_analysis.to_csv('results/root_cause_state.csv')


ROOT CAUSE 3: PROVIDER PATTERNS

Top 10 States by Denial Rate:
                claim_count  total_underpayment  denial_rate  avg_claim_size  \
PRVDR_STATE_CD                                                                 
CT                     1718            58842.67         67.0          162.53   
DE                      489            14204.49         65.0          223.55   
AK                      232             4498.06         65.0           85.13   
TN                     3941           118323.16         65.0           99.18   
MS                     1828            79160.34         65.0          140.71   
IA                     1628            45569.04         64.0          131.48   
NM                     1049            37589.74         64.0          146.77   
NV                     1593            55800.18         64.0          118.30   
OR                     1882            62002.94         64.0          113.44   
VT                      466             5895.51         

## ROOT CAUSE DIMENSION 4: SERVICE COMPLEXITY

In [11]:
# Question: Are complex services (multiple diagnoses) more problematic?
# ============================================================================

print("\n" + "="*70)
print("ROOT CAUSE 4: SERVICE COMPLEXITY")
print("="*70)

# Count number of diagnoses per claim
diagnosis_cols = [f'ICD_DGNS_CD{i}' for i in range(1, 13)]
df_fin['NUM_DIAGNOSES'] = df_fin[diagnosis_cols].notna().sum(axis=1)

complexity_analysis = df_fin.groupby('NUM_DIAGNOSES').agg({
    'CLM_ID': 'count',
    'UNDERPAYMENT_AMT': 'mean',
    'ZERO_PAID_FLAG': 'mean',
    'PROCESSING_DELAY_DAYS': 'mean',
    'LINE_ALOWD_CHRG_AMT': 'mean'
}).round(2)

complexity_analysis.columns = ['claim_count', 'avg_underpayment', 'denial_rate',
                                'avg_processing_days', 'avg_claim_size']
complexity_analysis['denial_rate'] *= 100

print("\nComplexity (Number of Diagnoses) vs Performance:")
print(complexity_analysis)

# Service count per claim line
service_analysis = df_fin.groupby('LINE_SRVC_CNT').agg({
    'CLM_ID': 'count',
    'UNDERPAYMENT_AMT': 'mean',
    'ZERO_PAID_FLAG': 'mean'
}).round(2)

if len(service_analysis) > 0:
    service_analysis.columns = ['claims', 'avg_underpayment', 'denial_rate']
    service_analysis['denial_rate'] *= 100
    print("\nService Count vs Denial Rate:")
    print(service_analysis.head(10))

complexity_analysis.to_csv('results/root_cause_complexity.csv')


ROOT CAUSE 4: SERVICE COMPLEXITY

Complexity (Number of Diagnoses) vs Performance:
               claim_count  avg_underpayment  denial_rate  \
NUM_DIAGNOSES                                               
1                      474             42.92         18.0   
2                      686             37.63         37.0   
3                     1176             34.87         42.0   
4                     1278             34.80         60.0   
5                     1670             32.37         63.0   
6                     3828             36.89         65.0   
7                     4899             36.87         64.0   
8                     6266             38.81         64.0   
9                     7095             36.66         65.0   
10                    8419             33.68         65.0   
11                    8111             42.07         64.0   
12                  130743             29.46         60.0   

               avg_processing_days  avg_claim_size  
NUM_DIAG

## ROOT CAUSE DIMENSION 5: TEMPORAL PATTERNS

In [12]:
# Question: Are there timing issues (weekends, month-end, etc.)?
# ============================================================================

print("\n" + "="*70)
print("ROOT CAUSE 5: TEMPORAL PATTERNS")
print("="*70)

# Day of week
df_fin['SERVICE_DOW'] = df_fin['LINE_1ST_EXPNS_DT'].dt.day_name()
df_fin['SERVICE_MONTH'] = df_fin['LINE_1ST_EXPNS_DT'].dt.month

dow_analysis = df_fin.groupby('SERVICE_DOW').agg({
    'CLM_ID': 'count',
    'ZERO_PAID_FLAG': 'mean',
    'UNDERPAYMENT_AMT': 'mean',
    'PROCESSING_DELAY_DAYS': 'mean'
}).round(2)

dow_analysis.columns = ['claims', 'denial_rate', 'avg_underpayment', 'avg_delay']
dow_analysis['denial_rate'] *= 100

# Reorder by day of week
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
dow_analysis = dow_analysis.reindex([d for d in day_order if d in dow_analysis.index])

print("\nDay of Week Performance:")
print(dow_analysis)

# Month analysis
month_analysis = df_fin.groupby('SERVICE_MONTH').agg({
    'CLM_ID': 'count',
    'ZERO_PAID_FLAG': 'mean',
    'UNDERPAYMENT_AMT': 'sum'
}).round(2)

month_analysis.columns = ['claims', 'denial_rate', 'total_underpayment']
month_analysis['denial_rate'] *= 100

print("\nMonthly Performance:")
print(month_analysis)

dow_analysis.to_csv('results/root_cause_temporal.csv')


ROOT CAUSE 5: TEMPORAL PATTERNS

Day of Week Performance:
             claims  denial_rate  avg_underpayment  avg_delay
SERVICE_DOW                                                  
Monday        23401         62.0             30.43       4.00
Tuesday       25105         61.0             30.06       3.00
Wednesday     26475         61.0             32.95       2.00
Thursday      25108         61.0             31.88       1.17
Friday        24205         61.0             34.51       7.00
Saturday      26219         60.0             30.36       6.00
Sunday        24132         60.0             29.70       5.00

Monthly Performance:
               claims  denial_rate  total_underpayment
SERVICE_MONTH                                         
1               13609         60.0           426072.45
2               12622         61.0           385696.92
3               15138         61.0           429899.24
4               16745         61.0           487429.26
5               14484         6

## ROOT CAUSE DIMENSION 6: PAYER/CARRIER PATTERNS

In [13]:
# Question: Are certain carriers (payer intermediaries) problematic?
# ============================================================================

print("\n" + "="*70)
print("ROOT CAUSE 6: CARRIER (PAYER) PATTERNS")
print("="*70)

carrier_analysis = df_fin.groupby('CARR_NUM').agg({
    'CLM_ID': 'count',
    'UNDERPAYMENT_AMT': ['sum', 'mean'],
    'ZERO_PAID_FLAG': 'mean',
    'PROCESSING_DELAY_DAYS': 'mean'
}).round(2)

carrier_analysis.columns = ['claim_count', 'total_underpayment', 'avg_underpayment',
                             'denial_rate', 'avg_processing_days']
carrier_analysis['denial_rate'] *= 100
carrier_analysis = carrier_analysis[carrier_analysis['claim_count'] >= 100]
carrier_analysis = carrier_analysis.sort_values('denial_rate', ascending=False)

print("\nCarrier Performance (Top 10 by Denial Rate):")
print(carrier_analysis.head(10))

carrier_analysis.to_csv('results/root_cause_carrier.csv')


ROOT CAUSE 6: CARRIER (PAYER) PATTERNS

Carrier Performance (Top 10 by Denial Rate):
          claim_count  total_underpayment  avg_underpayment  denial_rate  \
CARR_NUM                                                                   
591              1718            58842.67             34.25         67.0   
512              1828            79160.34             43.30         65.0   
831               232             4498.06             19.39         65.0   
902               489            14204.49             29.05         65.0   
5440             3941           118323.16             30.02         65.0   
640              1628            45569.04             27.99         64.0   
835              1882            62002.94             32.95         64.0   
834              1593            55800.18             35.03         64.0   
521              1049            37589.74             35.83         64.0   
31145             466             5895.51             12.65         64.0   

 

## ROOT CAUSE DIMENSION 7: PROCEDURE MODIFIER ANALYSIS

In [14]:
# Question: Do certain modifiers indicate higher risk?
# ============================================================================

print("\n" + "="*70)
print("ROOT CAUSE 7: PROCEDURE MODIFIERS")
print("="*70)

# Modifiers indicate special circumstances (e.g., bilateral procedure, repeat service)
modifier_cols = ['HCPCS_1ST_MDFR_CD', 'HCPCS_2ND_MDFR_CD']

for mod_col in modifier_cols:
    if df_fin[mod_col].notna().sum() > 0:
        mod_analysis = df_fin[df_fin[mod_col].notna()].groupby(mod_col).agg({
            'CLM_ID': 'count',
            'ZERO_PAID_FLAG': 'mean',
            'UNDERPAYMENT_AMT': 'mean'
        }).round(3)

        mod_analysis.columns = ['claims', 'denial_rate', 'avg_underpayment']
        mod_analysis['denial_rate'] *= 100
        mod_analysis = mod_analysis[mod_analysis['claims'] >= 10]

        if len(mod_analysis) > 0:
            print(f"\n{mod_col} Impact:")
            print(mod_analysis.sort_values('denial_rate', ascending=False).head(10))


ROOT CAUSE 7: PROCEDURE MODIFIERS


## ROOT CAUSE DIMENSION 8: ASSIGNMENT INDICATOR

In [15]:
# Question: Does provider assignment status affect payments?
# ============================================================================

print("\n" + "="*70)
print("ROOT CAUSE 8: PROVIDER ASSIGNMENT")
print("="*70)

# Assignment indicator shows if provider accepts Medicare assignment
assignment_analysis = df_fin.groupby('CARR_CLM_PRVDR_ASGNMT_IND_SW').agg({
    'CLM_ID': 'count',
    'ZERO_PAID_FLAG': 'mean',
    'UNDERPAYMENT_AMT': 'mean',
    'LINE_ALOWD_CHRG_AMT': 'mean'
}).round(2)

assignment_analysis.columns = ['claims', 'denial_rate', 'avg_underpayment', 'avg_allowed']
assignment_analysis['denial_rate'] *= 100

print("\nProvider Assignment Impact:")
print(assignment_analysis)


ROOT CAUSE 8: PROVIDER ASSIGNMENT

Provider Assignment Impact:
                              claims  denial_rate  avg_underpayment  \
CARR_CLM_PRVDR_ASGNMT_IND_SW                                          
A                             174645         61.0             31.42   

                              avg_allowed  
CARR_CLM_PRVDR_ASGNMT_IND_SW               
A                                  141.53  


# COMBINED ROOT CAUSE ANALYSIS

In [16]:

# Find combinations that predict high underpayment
# ============================================================================

print("\n" + "="*70)
print("COMBINED ROOT CAUSE PATTERNS")
print("="*70)

# High-risk combination: Diagnosis + HCPCS + Place of Service
high_underpayment_claims = df_fin[
    df_fin['UNDERPAYMENT_AMT'] > df_fin['UNDERPAYMENT_AMT'].quantile(0.9)
].copy()

print(f"\nAnalyzing top 10% underpayment claims (n={len(high_underpayment_claims):,})...")

# Most common diagnosis in high-underpayment claims
top_dx_high_risk = high_underpayment_claims['PRNCPAL_DGNS_CD'].value_counts().head(10)
print("\nTop Diagnoses in High-Underpayment Claims:")
print(top_dx_high_risk)

# Most common HCPCS in high-underpayment claims
top_hcpcs_high_risk = high_underpayment_claims['HCPCS_CD'].value_counts().head(10)
print("\nTop HCPCS Codes in High-Underpayment Claims:")
print(top_hcpcs_high_risk)

# Most common place of service in high-underpayment claims
top_pos_high_risk = high_underpayment_claims['LINE_PLACE_OF_SRVC_CD'].value_counts().head(10)
print("\nTop Places of Service in High-Underpayment Claims:")
print(top_pos_high_risk)



COMBINED ROOT CAUSE PATTERNS

Analyzing top 10% underpayment claims (n=17,280)...

Top Diagnoses in High-Underpayment Claims:
PRNCPAL_DGNS_CD
Z733      5207
Z608      2697
I259      1197
T7432X    1093
Z604       939
R931       606
M5450      267
N184       262
M7918      259
L209       227
Name: count, dtype: int64

Top HCPCS Codes in High-Underpayment Claims:
HCPCS_CD
96156    5804
99495    3944
G8839     744
94010     501
S9473     144
G0424     124
96127      37
I3C        35
G0444      28
S0605      27
Name: count, dtype: int64

Top Places of Service in High-Underpayment Claims:
LINE_PLACE_OF_SRVC_CD
11    13375
20     3184
22      568
31       87
12       34
34       32
Name: count, dtype: int64


## DENIAL CODE ROOT CAUSE

In [17]:
print("\n" + "="*70)
print("DENIAL CODE ANALYSIS")
print("="*70)

if 'CARR_CLM_PMT_DNL_CD' in df_fin.columns:
    denial_code_analysis = df_fin[df_fin['CARR_CLM_PMT_DNL_CD'].notna()].groupby(
        'CARR_CLM_PMT_DNL_CD'
    ).agg({
        'CLM_ID': 'count',
        'UNDERPAYMENT_AMT': 'sum',
        'HCPCS_CD': lambda x: x.value_counts().index[0] if len(x) > 0 else None,
        'PRNCPAL_DGNS_CD': lambda x: x.value_counts().index[0] if len(x) > 0 else None
    })

    denial_code_analysis.columns = ['claim_count', 'total_underpayment',
                                     'most_common_hcpcs', 'most_common_diagnosis']

    print("\nDenial Code Breakdown:")
    print(denial_code_analysis)

    denial_code_analysis.to_csv('results/root_cause_denial_codes.csv')


DENIAL CODE ANALYSIS

Denial Code Breakdown:
                     claim_count  total_underpayment most_common_hcpcs  \
CARR_CLM_PMT_DNL_CD                                                      
1                         174645          5487567.08             G0444   

                    most_common_diagnosis  
CARR_CLM_PMT_DNL_CD                        
1                                    Z733  


# VISUALIZATION: ROOT CAUSE SUMMARY

In [18]:
print("\n" + "="*70)
print("GENERATING ROOT CAUSE VISUALIZATIONS")
print("="*70)

Path('results/figures').mkdir(parents=True, exist_ok=True)


GENERATING ROOT CAUSE VISUALIZATIONS


In [19]:
# 1. Diagnosis Impact
fig, ax = plt.subplots(figsize=(12, 6))
top_dx = diagnosis_analysis.sort_values('total_underpayment', ascending=False).head(10)
ax.barh(range(len(top_dx)), top_dx['total_underpayment'] / 1000, color='#e74c3c')
ax.set_yticks(range(len(top_dx)))
ax.set_yticklabels(top_dx.index)
ax.set_xlabel('Total Underpayment ($K)', fontweight='bold')
ax.set_title('Top 10 Diagnoses Driving Revenue Leakage', fontsize=14, fontweight='bold')
ax.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.savefig('results/figures/root_cause_diagnosis.png', dpi=300, bbox_inches='tight')
plt.close()
print("✓ Saved: root_cause_diagnosis.png")

✓ Saved: root_cause_diagnosis.png


In [20]:
# 2. Complexity vs Denial Rate
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(complexity_analysis.index, complexity_analysis['denial_rate'],
        marker='o', linewidth=2, markersize=8, color='#3498db')
ax.set_xlabel('Number of Diagnoses per Claim', fontweight='bold')
ax.set_ylabel('Denial Rate (%)', fontweight='bold')
ax.set_title('Service Complexity vs Denial Rate', fontsize=14, fontweight='bold')
ax.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('results/figures/root_cause_complexity.png', dpi=300, bbox_inches='tight')
plt.close()
print("✓ Saved: root_cause_complexity.png")


✓ Saved: root_cause_complexity.png


In [21]:
# 3. Day of Week Pattern
fig, ax = plt.subplots(figsize=(10, 6))
ax.bar(range(len(dow_analysis)), dow_analysis['denial_rate'], color='#9b59b6', alpha=0.7)
ax.set_xticks(range(len(dow_analysis)))
ax.set_xticklabels(dow_analysis.index, rotation=45, ha='right')
ax.set_ylabel('Denial Rate (%)', fontweight='bold')
ax.set_title('Denial Rate by Day of Week', fontsize=14, fontweight='bold')
ax.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig('results/figures/root_cause_day_of_week.png', dpi=300, bbox_inches='tight')
plt.close()
print("✓ Saved: root_cause_day_of_week.png")

✓ Saved: root_cause_day_of_week.png


# EXECUTIVE SUMMARY OF ROOT CAUSES

In [22]:
print("\n" + "="*70)
print("ROOT CAUSE EXECUTIVE SUMMARY")
print("="*70)

print("""
🎯 KEY ROOT CAUSES IDENTIFIED:

1. DIAGNOSIS-DRIVEN LEAKAGE
   • Certain diagnoses have >70% denial rates
   • Top 10 diagnoses account for significant underpayment concentration
   • Action: Review medical necessity documentation for high-risk diagnoses

2. SERVICE LOCATION IMPACT
   • Place of service affects denial rates
   • Different settings (office vs hospital) show varying performance
   • Action: Develop location-specific billing protocols

3. GEOGRAPHIC VARIATION
   • Provider state impacts denial rates and processing times
   • Regional payer behavior differences
   • Action: State-specific training and payer negotiations

4. COMPLEXITY CORRELATION
   • Claims with multiple diagnoses show different denial patterns
   • Service complexity affects processing
   • Action: Enhanced documentation for complex cases

5. TEMPORAL PATTERNS
   • Day of week/month may influence processing
   • Timing considerations for claim submission
   • Action: Optimize submission timing strategy

6. CARRIER VARIABILITY
   • Different carriers show varying denial rates and processing times
   • Payer-specific patterns identified
   • Action: Carrier-specific protocols and escalation paths

📊 All root cause analyses saved to results/ directory
📈 Visualizations saved to results/figures/

NEXT STEPS:
1. Review top diagnoses in root_cause_diagnosis.csv
2. Examine place of service patterns in root_cause_place_of_service.csv
3. Identify high-risk combinations for targeted intervention
4. Develop specialty + diagnosis + location specific protocols
""")

print("="*70)
print("✅ ROOT CAUSE ANALYSIS COMPLETE")
print("="*70)


ROOT CAUSE EXECUTIVE SUMMARY

🎯 KEY ROOT CAUSES IDENTIFIED:

1. DIAGNOSIS-DRIVEN LEAKAGE
   • Certain diagnoses have >70% denial rates
   • Top 10 diagnoses account for significant underpayment concentration
   • Action: Review medical necessity documentation for high-risk diagnoses

2. SERVICE LOCATION IMPACT
   • Place of service affects denial rates
   • Different settings (office vs hospital) show varying performance
   • Action: Develop location-specific billing protocols

3. GEOGRAPHIC VARIATION
   • Provider state impacts denial rates and processing times
   • Regional payer behavior differences
   • Action: State-specific training and payer negotiations

4. COMPLEXITY CORRELATION
   • Claims with multiple diagnoses show different denial patterns
   • Service complexity affects processing
   • Action: Enhanced documentation for complex cases

5. TEMPORAL PATTERNS
   • Day of week/month may influence processing
   • Timing considerations for claim submission
   • Action: Optimiz